In [2]:
import requests
import os
import pandas as pd
from xml.dom import minidom
import xml.etree.ElementTree as et
import numpy as np

df_rows = []
df_columns = ["itemid", "text", "bip:topics"]
files_path = (r'C:\Users\gouth\OneDrive\Desktop\Dalhousie\ML BD\Assignments\Assignment 1\Data\New folder\New folder')
files = []
column_bip_topics = []
for r, d, f in os.walk(files_path):
    for file in f:
        if '.xml' in file:
            files.append(os.path.join(r,file))
#-----------------------Get Text and bip values from all files--------------------------------------------------------------
def GetDataFrame(files): 
    for name in files:
        address = name.split('\\')
        
        xml_bip_topics = ''
        xml_text = ''
        dom = et.parse(name)
        root = dom.getroot()
        xml_itemId = root.attrib.get("itemid") ##GET ITEM ID
        
        for child in root:
                if(child.tag == 'text'):
                    for textnode in child:
                        if(textnode.tag == 'p'):
                            #print(dir(textnode))
                            xml_text = xml_text + " " + textnode.text##GET TEXT
                for j in child.iter('metadata'):
                    for bip in j.iterfind(".//codes[@class='bip:topics:1.0']"):
                            for bipnodes in bip.iter("code"):
                                xml_bip_topics = bipnodes.attrib.get("code") + "," + xml_bip_topics ##GET BIP_CODES
                    
        column_bip_topics.append(xml_bip_topics)
        df_rows.append({"text" : xml_text, "bip:topics" : xml_bip_topics, "itemid" : xml_itemId })
    Final_DF = pd.DataFrame(df_rows, columns = df_columns)
    return Final_DF
Final_DF_Values = GetDataFrame(files)
Text_Column = (Final_DF_Values.loc[:,['text']]).values
#---------------------------------Extract unique Bip Values------------------------------------
def GetAllBipTopics(column_bip_topics):
    all_bips = []
    first_bip = []
    for b in column_bip_topics:
        indiv_bips = str(b).split(",")
        for bip in indiv_bips:
            all_bips.append(bip)
        first_bip.append(indiv_bips[0])
    All_Indivitual_bips = set(all_bips)
    All_Indivitual_bips = list(filter(None,All_Indivitual_bips)) ##All possible bip values
    return (All_Indivitual_bips,first_bip)
All_Indivitual_bips,first_bip = GetAllBipTopics(column_bip_topics)
for i in range(len(first_bip)):
    if len(first_bip[i]) == 0:
        first_bip[i] = 'NA'
#---------------------------------------Data Preprocessing Step------------------------------------------------
import nltk
#nltk.download('stopwords')
#nltk.download('punkt')
#nltk.download('wordnet')
from nltk.corpus import stopwords, wordnet
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
import re
import sys
def GetFiltedText(Text_Column):
    WL = WordNetLemmatizer()
    PS = PorterStemmer()
    StopWords = set(stopwords.words('english'))
    Filtered_WordList = []
    for text in Text_Column:
        FilterNumber1 = (re.sub('[^A-Za-z ]+', '', str(text))).lower() ## To lower and Selecting only alphabets
        Tokens = word_tokenize(FilterNumber1)
        FilterNumber2 = []
        Filtered_Sentence = ""
        for word in Tokens:
            if word not in StopWords:
                FilterNumber2.append(PS.stem(WL.lemmatize(word))) ##Stemming, lemmatizing and removing stop words
        for i in FilterNumber2:
            if(len(i) != 1 and len(i) != 2):
                Filtered_Sentence = Filtered_Sentence + " " + i
        Filtered_WordList.append(Filtered_Sentence)
    return Filtered_WordList
Filtered_WordList = GetFiltedText(Text_Column)
print(len(Filtered_WordList))
print(len(first_bip))
#----------------------------------------Using TF-IDF----------------------------------------------
from sklearn.feature_extraction.text import TfidfVectorizer as TV
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
def FeatureSelection(Filtered_WordList,first_bip):
    model = TV(max_features = 1800) ##Extract more frequency tokens first
    tfidf = model.fit_transform(Filtered_WordList)
    final_features = tfidf.toarray()
    return final_features
final_features = FeatureSelection(Filtered_WordList, first_bip)
#---------------------------------------Silhouette Score Calculation----------------------------------
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
print(__doc__)
range_n_clusters = [4,5,6]
for n_clusters in range_n_clusters:
    clusterer = KMeans(n_clusters=n_clusters, random_state=10)
    cluster_labels = clusterer.fit_predict(final_features)
    silhouette_avg = silhouette_score(final_features, cluster_labels, metric='euclidean')
    print("For n_clusters =", n_clusters,
          "The average silhouette_score is :", silhouette_avg)
#-------------------------------------------Clustering dataset by Silhouette value----------------------
One_big_matrix = []
One_big_matrix = np.column_stack((cluster_labels,final_features.astype(np.object),first_bip))
One_big_matrix = One_big_matrix[One_big_matrix[:,0].argsort()]
#One_big_matrix[:,1:-1] = One_big_matrix[:,1:-1].astype(np.float64)
a0 = np.count_nonzero(cluster_labels == 0)
a1 = a0 + np.count_nonzero(cluster_labels == 1)
a2 = a1 + np.count_nonzero(cluster_labels == 2)
a3 = a2 + np.count_nonzero(cluster_labels == 3)
a4 = a3 + np.count_nonzero(cluster_labels == 4)
a5 = a4 + np.count_nonzero(cluster_labels == 5)
First_Cluster = One_big_matrix[0:a0,:]
Second_Cluster = One_big_matrix[a0:a1,:]
Third_Cluster = One_big_matrix[a1:a2,:]
Fourth_Cluster = One_big_matrix[a2:a3,:]
Fifth_Cluster = One_big_matrix[a3:a4,:]
Sixth_Cluster = One_big_matrix[a4:a5,:]
#------------------------------------------------------------------------------------------------


48375
48375
Automatically created module for IPython interactive environment
For n_clusters = 4 The average silhouette_score is : 0.01136250615616302
For n_clusters = 5 The average silhouette_score is : 0.013225579609507997
For n_clusters = 6 The average silhouette_score is : 0.015675860712247067


In [9]:
from sklearn import metrics, model_selection
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier as mlp
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(First_Cluster[:,1:-1], First_Cluster[:,-1], test_size=0.3)
mlp1 = mlp(hidden_layer_sizes=(30,30,30))
mlp1.fit(X_train,y_train)
y_pred = mlp1.predict(X_test)
from sklearn.metrics import f1_score
print('Accuracy of First cluster: ' ,accuracy_score(y_test, y_pred))
#-------------------------------------------------------------------
from sklearn.ensemble import RandomForestClassifier as rf
X_train, X_test, y_train, y_test = train_test_split(Second_Cluster[:,1:-1], Second_Cluster[:,-1], test_size=0.3)
RF = rf(n_estimators=100, max_depth=2, random_state=0)
RF.fit(X_train, y_train)
y_pred = RF.predict(X_test)
print('Accuracy of Second cluster: ' ,accuracy_score(y_test, y_pred))
#--------------------------------------------------------------------
from sklearn import svm
X_train, X_test, y_train, y_test = train_test_split(Third_Cluster[:,1:-1], Third_Cluster[:,-1], test_size=0.3)
SVM = svm.LinearSVC()
SVM.fit(X_train, y_train)
y_pred = SVM.predict(X_test)
print('Accuracy of Third cluster: ' ,accuracy_score(y_test, y_pred))
#----------------------------------------------------------------------
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
X_train, X_test, y_train, y_test = train_test_split(Fourth_Cluster[:,1:-1], Fourth_Cluster[:,-1], test_size=0.3)
clf = DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print('Accuracy of Fourth cluster: ' ,accuracy_score(y_test, y_pred))
#------------------------------------------------------------------------
from sklearn.linear_model import LogisticRegression
X_train, X_test, y_train, y_test = train_test_split(Fifth_Cluster[:,1:-1], Fifth_Cluster[:,-1], test_size=0.3)
LR = LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr').fit(X_train, y_train)
y_pred = LR.predict(X_test)
print('Accuracy of Fifth cluster: ' ,accuracy_score(y_test, y_pred))
#----------------------------------------------------------------------
from sklearn.linear_model import SGDClassifier
X_train, X_test, y_train, y_test = train_test_split(Sixth_Cluster[:,1:-1], Sixth_Cluster[:,-1], test_size=0.3)
clf = SGDClassifier(max_iter=1000, tol=1e-3)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print('Accuracy of Sixth cluster: ' ,accuracy_score(y_test, y_pred))

Accuracy of First cluster:  0.8917311124330756
Accuracy of Second cluster:  0.5595984943538268
Accuracy of Third cluster:  0.8969404186795491
Accuracy of Fourth cluster:  0.5617521800851755
Accuracy of Fifth cluster:  0.8990719257540604
Accuracy of Sixth cluster:  0.9794372294372294


In [16]:
One_big_matrix1 = []
One_big_matrix1 = np.vstack((cluster_labels,Filtered_WordList,first_bip))
One_big_matrix1 = (One_big_matrix1[One_big_matrix1[:,0].argsort()]).T
b0 = np.count_nonzero(cluster_labels == 0)
b1 = b0 + np.count_nonzero(cluster_labels == 1)
b2 = b1 + np.count_nonzero(cluster_labels == 2)
b3 = b2 + np.count_nonzero(cluster_labels == 3)
b4 = b3 + np.count_nonzero(cluster_labels == 4)
b5 = b4 + np.count_nonzero(cluster_labels == 5)
First_Cluster = One_big_matrix1[0:b0,:]
Second_Cluster = One_big_matrix1[b0:b1,:]
Third_Cluster = One_big_matrix1[b1:b2,:]
Fourth_Cluster = One_big_matrix1[b2:b3,:]
Fifth_Cluster = One_big_matrix1[b3:b4,:]
Sixth_Cluster = One_big_matrix1[b4:b5,:]
#-----------------------------------------------------------------------------------------------

In [26]:
#------------------------------------ FIRST CLUSTER---------------------------------
import gensim 
EMBEDDING_DIM = 100
model = gensim.models.Word2Vec(sentences=First_Cluster[:,1], size=EMBEDDING_DIM, window=5, workers=4, min_count=1)
words = list(model.wv.vocab)
fn = 'word2vec.txt'
model.wv.save_word2vec_format(fn, binary=False)
import os
embeddings_index = {}
f = open(os.path.join('', 'word2vec.txt'),  encoding = "utf-8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:])
    embeddings_index[word] = coefs
f.close()
import warnings
warnings.filterwarnings("ignore")
#------------------------------------------------------------------------------
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
tokenizer_obj = Tokenizer()
tokenizer_obj.fit_on_texts(First_Cluster[:,1])
sequences = tokenizer_obj.texts_to_sequences(First_Cluster[:,1])
word_index = tokenizer_obj.word_index
After_padding = pad_sequences(sequences, maxlen=1500)
print(After_padding.shape)
#-----------------------------------------------------------------------------------
tokenizer_obj11 = Tokenizer()
tokenizer_obj11.fit_on_texts(First_Cluster[:,2])
sequences1 = tokenizer_obj11.texts_to_sequences(First_Cluster[:,2])
sequences11 = []
#print(int(sequences1[0]))
for i in sequences1:
    for j in i:
        sequences11.append(j)
from keras.utils.np_utils import to_categorical
sequences11 = to_categorical(sequences11)
Network(After_padding, sequences11, word_index,embeddings_index) ##calling Neural Network

(5601, 1500)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 1500, 100)         700       
_________________________________________________________________
spatial_dropout1d_5 (Spatial (None, 1500, 100)         0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 64)                42240     
_________________________________________________________________
dense_5 (Dense)              (None, 47)                3055      
Total params: 45,995
Trainable params: 45,995
Non-trainable params: 0
_________________________________________________________________
None
Train on 3920 samples, validate on 1681 samples
Epoch 1/3
 - 162s - loss: 0.1029 - acc: 0.9787 - val_loss: 0.1028 - val_acc: 0.9787
Epoch 2/3
 - 156s - loss: 0.1029 - acc: 0.9787 - val_loss: 0.1027 - val_acc: 0.9787
Epoch 3/3
 - 164s - loss: 0.1028 - a

In [42]:
#------------------------------------Second Cluster----------------------------------------
model = gensim.models.Word2Vec(sentences=Second_Cluster[:,1], size=EMBEDDING_DIM, window=5, workers=4, min_count=1)
words = list(model.wv.vocab)
fn = 'word2vec.txt'
model.wv.save_word2vec_format(fn, binary=False)
embeddings_index = {}
f = open(os.path.join('', 'word2vec.txt'),  encoding = "utf-8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:])
    embeddings_index[word] = coefs
f.close()
tokenizer_obj = Tokenizer()
tokenizer_obj.fit_on_texts(Second_Cluster[:,1])
sequences = tokenizer_obj.texts_to_sequences(Second_Cluster[:,1])
word_index = tokenizer_obj.word_index
After_padding = pad_sequences(sequences, maxlen=1500)
tokenizer_obj11 = Tokenizer()
tokenizer_obj11.fit_on_texts(Second_Cluster[:,2])
sequences1 = tokenizer_obj11.texts_to_sequences(Second_Cluster[:,2])
sequences11 = []
for i in sequences1:
    for j in i:
        sequences11.append(j)
Network(After_padding, sequences11, word_index,embeddings_index) ##calling Neural Network

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_19 (Embedding)     (None, 1500, 100)         700       
_________________________________________________________________
spatial_dropout1d_19 (Spatia (None, 1500, 100)         0         
_________________________________________________________________
lstm_19 (LSTM)               (None, 64)                42240     
_________________________________________________________________
dense_18 (Dense)             (None, 56)                3640      
Total params: 46,580
Trainable params: 46,580
Non-trainable params: 0
_________________________________________________________________
None
Train on 5577 samples, validate on 2391 samples
Epoch 1/3
 - 130s - loss: 0.0896 - acc: 0.9821 - val_loss: 0.0893 - val_acc: 0.9821
Epoch 2/3
 - 129s - loss: 0.0895 - acc: 0.9821 - val_loss: 0.0893 - val_acc: 0.9821
Epoch 3/3
 - 135s - loss: 0.0894 - acc: 0.9821 - 

In [44]:
#------------------------------------Third Cluster----------------------------------------
model = gensim.models.Word2Vec(sentences=Third_Cluster[:,1], size=EMBEDDING_DIM, window=5, workers=4, min_count=1)
words = list(model.wv.vocab)
fn = 'word2vec.txt'
model.wv.save_word2vec_format(fn, binary=False)
embeddings_index = {}
f = open(os.path.join('', 'word2vec.txt'),  encoding = "utf-8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:])
    embeddings_index[word] = coefs
f.close()
tokenizer_obj = Tokenizer()
tokenizer_obj.fit_on_texts(Third_Cluster[:,1])
sequences = tokenizer_obj.texts_to_sequences(Third_Cluster[:,1])
word_index = tokenizer_obj.word_index
After_padding = pad_sequences(sequences, maxlen=1500)
#-----------------------------------------------------------------------------------
tokenizer_obj11 = Tokenizer()
tokenizer_obj11.fit_on_texts(Third_Cluster[:,2])
sequences1 = tokenizer_obj11.texts_to_sequences(Third_Cluster[:,2])
sequences11 = []
for i in sequences1:
    for j in i:
        sequences11.append(j)
from keras.utils.np_utils import to_categorical
sequences11 = to_categorical(sequences11)
EMBEDDING_DIM =100
VALIDATION_SPLIT = 0.2
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i > num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
Network(After_padding, sequences11, word_index,embeddings_index)#Calling neural network

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_21 (Embedding)     (None, 1500, 100)         700       
_________________________________________________________________
spatial_dropout1d_21 (Spatia (None, 1500, 100)         0         
_________________________________________________________________
lstm_21 (LSTM)               (None, 64)                42240     
_________________________________________________________________
dense_20 (Dense)             (None, 64)                4160      
Total params: 47,100
Trainable params: 47,100
Non-trainable params: 0
_________________________________________________________________
None
Train on 8692 samples, validate on 3726 samples
Epoch 1/3
 - 231s - loss: 0.0807 - acc: 0.9844 - val_loss: 0.0806 - val_acc: 0.9844
Epoch 2/3
 - 245s - loss: 0.0807 - acc: 0.9844 - val_loss: 0.0806 - val_acc: 0.9844
Epoch 3/3
 - 262s - loss: 0.0806 - acc: 0.9844 - 

In [46]:
#------------------------------------Fourth Cluster----------------------------------------
model = gensim.models.Word2Vec(sentences=Fourth_Cluster[:,1], size=EMBEDDING_DIM, window=5, workers=4, min_count=1)
words = list(model.wv.vocab)
fn = 'word2vec.txt'
model.wv.save_word2vec_format(fn, binary=False)
embeddings_index = {}
f = open(os.path.join('', 'word2vec.txt'),  encoding = "utf-8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:])
    embeddings_index[word] = coefs
f.close()
tokenizer_obj = Tokenizer()
tokenizer_obj.fit_on_texts(Fourth_Cluster[:,1])
sequences = tokenizer_obj.texts_to_sequences(Fourth_Cluster[:,1])
word_index = tokenizer_obj.word_index
After_padding = pad_sequences(sequences, maxlen=1500)
tokenizer_obj11 = Tokenizer()
tokenizer_obj11.fit_on_texts(Fourth_Cluster[:,2])
sequences1 = tokenizer_obj11.texts_to_sequences(Fourth_Cluster[:,2])
sequences11 = []

for i in sequences1:
    for j in i:
        sequences11.append(j)
from keras.utils.np_utils import to_categorical
sequences11 = to_categorical(sequences11)
from keras.utils.np_utils import to_categorical
sequences11 = to_categorical(sequences11)
EMBEDDING_DIM =100
VALIDATION_SPLIT = 0.2
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i > num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
Network(After_padding, sequences11, word_index,embeddings_index)#Calling neural network
#------------------------------------------------------------------------------------

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_23 (Embedding)     (None, 1500, 100)         700       
_________________________________________________________________
spatial_dropout1d_23 (Spatia (None, 1500, 100)         0         
_________________________________________________________________
lstm_23 (LSTM)               (None, 64)                42240     
_________________________________________________________________
dense_22 (Dense)             (None, 66)                4290      
Total params: 47,230
Trainable params: 47,230
Non-trainable params: 0
_________________________________________________________________
None
Train on 11505 samples, validate on 4931 samples
Epoch 1/3
 - 404s - loss: 0.0784 - acc: 0.9848 - val_loss: 0.0784 - val_acc: 0.9848
Epoch 2/3
 - 485s - loss: 0.0784 - acc: 0.9848 - val_loss: 0.0783 - val_acc: 0.9848
Epoch 3/3
 - 571s - loss: 0.0783 - acc: 0.9848 -

In [48]:
#------------------------------------Fifth Cluster----------------------------------------
model = gensim.models.Word2Vec(sentences=Fifth_Cluster[:,1], size=EMBEDDING_DIM, window=5, workers=4, min_count=1)
words = list(model.wv.vocab)
fn = 'word2vec.txt'
model.wv.save_word2vec_format(fn, binary=False)
embeddings_index = {}
f = open(os.path.join('', 'word2vec.txt'),  encoding = "utf-8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:])
    embeddings_index[word] = coefs
f.close()
tokenizer_obj = Tokenizer()
tokenizer_obj.fit_on_texts(Fifth_Cluster[:,1])
sequences = tokenizer_obj.texts_to_sequences(Fifth_Cluster[:,1])
word_index = tokenizer_obj.word_index
After_padding = pad_sequences(sequences, maxlen=1500)
#-----------------------------------------------------------------------------------
tokenizer_obj11 = Tokenizer()
tokenizer_obj11.fit_on_texts(Fifth_Cluster[:,2])
sequences1 = tokenizer_obj11.texts_to_sequences(Fifth_Cluster[:,2])
sequences11 = []

for i in sequences1:
    for j in i:
        sequences11.append(j)
from keras.utils.np_utils import to_categorical
sequences11 = to_categorical(sequences11)

EMBEDDING_DIM =100
VALIDATION_SPLIT = 0.2
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))

for word, i in word_index.items():
    if i > num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
Network(After_padding, sequences11, word_index,embeddings_index)#Calling neural network

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_25 (Embedding)     (None, 1500, 100)         700       
_________________________________________________________________
spatial_dropout1d_25 (Spatia (None, 1500, 100)         0         
_________________________________________________________________
lstm_25 (LSTM)               (None, 64)                42240     
_________________________________________________________________
dense_24 (Dense)             (None, 48)                3120      
Total params: 46,060
Trainable params: 46,060
Non-trainable params: 0
_________________________________________________________________
None
Train on 2010 samples, validate on 862 samples
Epoch 1/3
 - 120s - loss: 0.1016 - acc: 0.9792 - val_loss: 0.1015 - val_acc: 0.9792
Epoch 2/3
 - 111s - loss: 0.1016 - acc: 0.9792 - val_loss: 0.1015 - val_acc: 0.9792
Epoch 3/3
 - 114s - loss: 0.1015 - acc: 0.9792 - v

In [50]:
#------------------------------------Sixth Cluster------------------------------
model = gensim.models.Word2Vec(sentences=Sixth_Cluster[:,1], size=EMBEDDING_DIM, window=5, workers=4, min_count=1)
words = list(model.wv.vocab)
fn = 'word2vec.txt'
model.wv.save_word2vec_format(fn, binary=False)
embeddings_index = {}
f = open(os.path.join('', 'word2vec.txt'),  encoding = "utf-8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:])
    embeddings_index[word] = coefs
f.close()
tokenizer_obj = Tokenizer()
tokenizer_obj.fit_on_texts(Sixth_Cluster[:,1])
sequences = tokenizer_obj.texts_to_sequences(Sixth_Cluster[:,1])
word_index = tokenizer_obj.word_index
After_padding = pad_sequences(sequences, maxlen=1500)
#-----------------------------------------------------------------------------------
tokenizer_obj11 = Tokenizer()
tokenizer_obj11.fit_on_texts(Sixth_Cluster[:,2])
sequences1 = tokenizer_obj11.texts_to_sequences(Sixth_Cluster[:,2])
sequences11 = []
for i in sequences1:
    for j in i:
        sequences11.append(j)
from keras.utils.np_utils import to_categorical
sequences11 = to_categorical(sequences11)

EMBEDDING_DIM =100
VALIDATION_SPLIT = 0.2
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))

for word, i in word_index.items():
    if i > num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
Network(After_padding, sequences11, word_index,embeddings_index)#Calling neural network

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_27 (Embedding)     (None, 1500, 100)         700       
_________________________________________________________________
spatial_dropout1d_27 (Spatia (None, 1500, 100)         0         
_________________________________________________________________
lstm_27 (LSTM)               (None, 64)                42240     
_________________________________________________________________
dense_26 (Dense)             (None, 39)                2535      
Total params: 45,475
Trainable params: 45,475
Non-trainable params: 0
_________________________________________________________________
None
Train on 2156 samples, validate on 924 samples
Epoch 1/3
 - 138s - loss: 0.1196 - acc: 0.9744 - val_loss: 0.1194 - val_acc: 0.9744
Epoch 2/3
 - 127s - loss: 0.1195 - acc: 0.9744 - val_loss: 0.1194 - val_acc: 0.9744
Epoch 3/3
 - 137s - loss: 0.1196 - acc: 0.9744 - v

In [31]:
#------------------------------CNN using LSTM function-----------------------------------------------
def Network(ClusterInput, sequences11, word_index,embeddings_index, densesize):
    EMBEDDING_DIM =100
    VALIDATION_SPLIT = 0.2
    num_words = len(word_index) + 1
    embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))

    for word, i in word_index.items():
        if i > num_words:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    #-----------------------------------------------------------------------------------
    from keras.models import Sequential
    from keras.layers import Dense, Embedding, Flatten
    from keras.layers.convolutional import Conv1D
    from keras.layers.convolutional import MaxPooling1D
    from keras.initializers import Constant
    from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
    from keras.optimizers import SGD
    opt = SGD(lr=0.01)
    model = Sequential()
    embedding_layer = Embedding(num_words,
                                EMBEDDING_DIM,
                                input_length=1500,
                                trainable=True)

    model.add(embedding_layer)
    model.add(SpatialDropout1D(0.8))
    model.add(LSTM(64, dropout=0.7, recurrent_dropout=0.7))
    model.add(Dense(46, activation='softmax'))
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    print(model.summary())
    #-------------------------------------------------------------------------------------------
    
    X_train_pad, X_test_pad, y_train, y_test = train_test_split(ClusterInput, sequences11, test_size=0.3)
    dense_num = np.size(y_train,1)
    #------------------------------------------------------------------------------------------
    model.fit(X_train_pad, y_train, batch_size=128, epochs=3, validation_data=(X_test_pad, y_test), verbose=2)
    return